In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
connex = sqlite3.connect('database.sqlite')
cur = connex.cursor()

In [3]:
loanbook = pd.read_sql_query('select * from loan;', connex)
loanbook.describe()

,index,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,887383.000000,8.873820e+05,887382.000000,887382.000000,887382.000000,887382.000000,8.873780e+05,887382.00000,887353.000000,887353.000000,...,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,8.171060e+05,21372.000000,21372.000000,21372.000000
mean,152155.671062,3.500184e+07,14755.280082,14741.893147,14702.480039,436.717647,7.502761e+04,18.15702,0.314445,0.694622,...,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.206868e+04,0.943945,1.537058,1.977307
std,111089.623731,2.411336e+07,8435.456408,8429.898507,8442.107612,244.186619,6.469823e+04,17.19060,0.862247,0.998447,...,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.749833e+04,1.446872,2.723339,2.874067
min,0.000000,7.047300e+04,500.000000,500.000000,0.000000,15.670000,0.000000e+00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,59769.500000,1.087713e+07,8000.000000,8000.000000,8000.000000,260.710000,4.500000e+04,11.91000,0.000000,0.000000,...,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,133718.000000,3.709528e+07,13000.000000,13000.000000,13000.000000,382.550000,6.500000e+04,17.65000,0.000000,0.000000,...,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.370000e+04,0.000000,0.000000,2.000000
75%,217438.500000,5.847135e+07,20000.000000,20000.000000,20000.000000,572.600000,9.000000e+04,23.95000,0.000000,1.000000,...,47858.000000,87.600000,2.000000,4.000000,7771.500000,75.200000,3.980000e+04,1.000000,2.000000,3.000000
max,421094.000000,7.354484e+07,35000.000000,35000.000000,35000.000000,1445.460000,9.500000e+06,9999.00000,39.000000,33.000000,...,878459.000000,223.300000,22.000000,43.000000,127305.000000,151.400000,9.999999e+06,17.000000,35.000000,32.000000


In [4]:
d = {'Current': True, 'Fully Paid':True, 'Charged Off': False, 'Default':False, 'Late (31-120 days)':False, 'In Grace Period':False, 'Late (16-30 days)':False, 'Does not meet the credit policy. Status:Fully Paid':True, 'Does not meet the credit policy. Status:Charged Off':False, None:False, 'Issued':True }

In [5]:
loanbook['loan_status'] = loanbook['loan_status'].map(d)

In [6]:
lbook = loanbook[['dti','annual_inc','loan_status']]
lbook = lbook.dropna()
lbook.describe()

,dti,annual_inc
count,887378.000000,8.873780e+05
mean,18.157094,7.502761e+04
std,17.190604,6.469823e+04
min,0.000000,0.000000e+00
25%,11.910000,4.500000e+04
50%,17.650000,6.500000e+04
75%,23.950000,9.000000e+04
max,9999.000000,9.500000e+06


In [7]:
loans_x_train, loans_x_test, loans_y_train, loans_y_test = train_test_split(lbook[['dti','annual_inc']], lbook['loan_status'], test_size = .33, random_state=42)

In [8]:
clf = RandomForestClassifier(n_jobs=2)

In [9]:
clf.fit(loans_x_train, loans_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [10]:
clf.predict(loans_x_test)

array([False,  True,  True, ...,  True,  True,  True], dtype=bool)

In [11]:
clf.predict_proba(loans_x_test)[0:10]

array([[ 0.6       ,  0.4       ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.1       ,  0.9       ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.10471055,  0.89528945],
       [ 0.        ,  1.        ]])

In [20]:
loans_y_test.describe()

count     292835
unique         2
top         True
freq      270698
Name: loan_status, dtype: object

In [24]:
preds = loans_y_test[clf.predict(loans_x_test)]

In [26]:
preds.describe()

count     285226
unique         2
top         True
freq      263677
Name: loan_status, dtype: object

In [ ]:
loans